In [3]:
## Include mxnet path: you should include your mxnet local path, if mxnet path is global, 
## you don't need to include it.
import sys
sys.path.append('../../practice_demo')

import os
import mxnet as mx
import numpy as np
import copy
import re
from google.protobuf import text_format
import json
import cv2 as cv
import scipy
import PIL.Image
import math
import time
import scipy
import matplotlib
%matplotlib inline
import pylab as plt
from scipy.ndimage.filters import gaussian_filter
from generateLabelCPM import *
from modelCPM import *

from collections import namedtuple
Point = namedtuple('Point', 'x y')
crop_size_x = 368
crop_size_y = 368
center_perterb_max = 40

#use_caffe = True
scale_prob = 1
scale_min = 0.5
scale_max = 1.1
target_dist = 0.6

In [4]:
modelId = 1
# set this part
param = dict()
# GPU device number (doesn't matter for CPU mode)
GPUdeviceNumber = 0
# Select model (default: 5)
param['modelID'] = modelId
# Use click mode or not. If yes (1), you will be asked to click on the center
# of person to be pose-estimated (for multiple people image). If not (0),
# the model will simply be applies on the whole image.
param['click'] = 1
# Scaling paramter: starting and ending ratio of person height to image
# height, and number of scales per octave
# warning: setting too small starting value on non-click mode will take
# large memory
# CPU mode or GPU mode
param['use_gpu'] = 1
param['test_mode'] = 3
param['vis'] = 1
param['octave'] = 6
param['starting_range'] = 0.8
param['ending_range'] = 2
param['min_num'] = 4
param['mid_num'] = 10
# the larger the crop_ratio, the smaller the windowsize
param['crop_ratio'] = 2.5  # 2
param['bbox_ratio'] = 0.25 # 0.5
# applyModel_max
param['max'] = 0
# use average heatmap
param['merge'] = 'avg'

# path of your caffe
caffepath = '/home/zhecao/caffe/matlab';

if modelId == 1:
    param['scale_search'] = [0.5, 1, 1.5, 2]
    param['thre1'] = 0.1
    param['thre2'] = 0.05 
    param['thre3'] = 0.5 

    param['model'] = dict()
    param['model'][1] = dict()
    param['model'][1]['caffemodel'] = '../model/_trained_COCO/pose_iter_440000.caffemodel'
    param['model'][1]['deployFile'] = '../model/_trained_COCO/pose_deploy.prototxt'
    param['model'][1]['description'] = 'COCO Pose56 Two-level Linevec'
    param['model'][1]['boxsize'] = 368
    param['model'][1]['padValue'] = 128
    param['model'][1]['np'] = 18
    param['model'][1]['part_str'] = ['nose', 'neck', 'Rsho', 'Relb', 'Rwri', 
                             'Lsho', 'Lelb', 'Lwri', 
                             'Rhip', 'Rkne', 'Rank',
                             'Lhip', 'Lkne', 'Lank',
                             'Leye', 'Reye', 'Lear', 'Rear', 'pt19']
if modelId == 2:
    param['scale_search'] = [0.7, 1, 1.3]
    param['thre1'] = 0.05
    param['thre2'] = 0.01 
    param['thre3'] = 3
    param['thre4'] = 0.1

    param['model'] = dict()
    param['model'][2] = dict()
    param['model'][2]['caffemodel'] = '../model/_trained_MPI/pose_iter_146000.caffemodel'
    param['model'][2]['deployFile'] = '../model/_trained_MPI/pose_deploy.prototxt'
    param['model'][2]['description'] = 'COCO Pose56 Two-level Linevec'
    param['model'][2]['boxsize'] = 368
    param['model'][2]['padValue'] = 128
    param['model'][2]['np'] = 18
    param['model'][2]['part_str'] = ['nose', 'neck', 'Rsho', 'Relb', 'Rwri',  
                                     'Lsho', 'Lelb', 'Lwri', 
                                     'Rhip', 'Rkne', 'Rank', 
                                     'Lhip', 'Lkne', 'Lank', 
                                     'Leye', 'Reye', 'Lear', 'Rear', 'pt19']

In [5]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
from numpy import ma
import skimage.io as io
import matplotlib.pyplot as plt
import matplotlib
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)
annFile = '/data/guest_users/liangdong/liangdong/practice_demo/coco/person_keypoints_trainval2014/person_keypoints_val2014.json'

coco = COCO(annFile)
cats = coco.loadCats(coco.getCatIds())
catIds = coco.getCatIds(catNms=['person'])
imgIds = coco.getImgIds(catIds=catIds )

loading annotations into memory...
Done (t=2.46s)
creating index...
index created!


In [6]:
def padHeight(img, padValue, bbox):
    h = img.shape[0]
    w = img.shape[1]
    h = min(bbox[0], h)
    
    print 'box before'
    print bbox
    bbox[0] = int(math.ceil(bbox[0]/8.0)*8)
    bbox[1] = int(max(bbox[1], w))
    bbox[1] = int(math.ceil(bbox[1]/8.0)*8)
    print 'box after'
    print bbox
    
    pad = 4 * [None]
    pad[0] = int(np.floor((bbox[0]-h)/2)) # up
    pad[1] = int(np.floor((bbox[1]-w)/2)) # left
    pad[2] = int(bbox[0]-h-pad[0]) # down
    pad[3] = int(bbox[1]-w-pad[1]) # right
    
    print pad
    
    img_padded = img
    
    print img_padded.shape
    pad_up = np.tile(img_padded[0:1,:,:]*0 + padValue, (pad[0], 1, 1))
    img_padded = np.concatenate((pad_up, img_padded), axis=0)
    pad_left = np.tile(img_padded[:,0:1,:]*0 + padValue, (1, pad[1], 1))
    img_padded = np.concatenate((pad_left, img_padded), axis=1)
    pad_down = np.tile(img_padded[-2:-1,:,:]*0 + padValue, (pad[2], 1, 1))
    img_padded = np.concatenate((img_padded, pad_down), axis=0)
    pad_right = np.tile(img_padded[:,-2:-1,:]*0 + padValue, (1, pad[3], 1))
    img_padded = np.concatenate((img_padded, pad_right), axis=1)

    return img_padded, pad
    # cropping if needed

In [7]:
def padRightDownCorner(img, stride, padValue):
    h = img.shape[0]
    w = img.shape[1]

    pad = 4 * [None]
    pad[0] = 0 # up
    pad[1] = 0 # left
    pad[2] = 0 if (h%stride==0) else stride - (h % stride) # down
    pad[3] = 0 if (w%stride==0) else stride - (w % stride) # right

    img_padded = img
    pad_up = np.tile(img_padded[0:1,:,:]*0 + padValue, (pad[0], 1, 1))
    img_padded = np.concatenate((pad_up, img_padded), axis=0)
    pad_left = np.tile(img_padded[:,0:1,:]*0 + padValue, (1, pad[1], 1))
    img_padded = np.concatenate((pad_left, img_padded), axis=1)
    pad_down = np.tile(img_padded[-2:-1,:,:]*0 + padValue, (pad[2], 1, 1))
    img_padded = np.concatenate((img_padded, pad_down), axis=0)
    pad_right = np.tile(img_padded[:,-2:-1,:]*0 + padValue, (1, pad[3], 1))
    img_padded = np.concatenate((img_padded, pad_right), axis=1)

    return img_padded, pad

class DataBatch(object):
    def __init__(self, data, label, pad=0):
        self.data = [data]
        self.label = [label]
        self.pad = pad

In [9]:
sym, _, _ = mx.model.load_checkpoint('../realtimePose', 0)
output_prefix='testConfigModel'
sym1, arg_params, aux_params = mx.model.load_checkpoint(output_prefix, 6)

MXNetError: [17:24:27] src/io/local_filesys.cc:154: Check failed: allow_null  LocalFileSystem: fail to open "testConfigModel-symbol.json"

Stack trace returned 10 entries:
[bt] (0) /data/guest_users/liangdong/liangdong/practice_demo/mxnet/libmxnet.so(_ZN4dmlc2io15LocalFileSystem4OpenERKNS0_3URIEPKcb+0x640) [0x7f8cafad5b30]
[bt] (1) /data/guest_users/liangdong/liangdong/practice_demo/mxnet/libmxnet.so(_ZN4dmlc6Stream6CreateEPKcS2_b+0x1c6) [0x7f8cafac7386]
[bt] (2) /data/guest_users/liangdong/liangdong/practice_demo/mxnet/libmxnet.so(MXSymbolCreateFromFile+0x66) [0x7f8cafa2c006]
[bt] (3) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call_unix64+0x4c) [0x7f8cf7668e40]
[bt] (4) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call+0x2eb) [0x7f8cf76688ab]
[bt] (5) /usr/lib/python2.7/lib-dynload/_ctypes.x86_64-linux-gnu.so(_ctypes_callproc+0x48f) [0x7f8cf78783df]
[bt] (6) /usr/lib/python2.7/lib-dynload/_ctypes.x86_64-linux-gnu.so(+0x11d82) [0x7f8cf787cd82]
[bt] (7) /usr/bin/python(PyObject_Call+0x43) [